In [9]:
from torchvision.models import resnet50, ResNet50_Weights
import torch.nn as nn 
import torch.nn.functional as F
import torch.optim as optim 
from torchvision import datasets, transforms
from torch.utils.data import Subset
import torch
import os
import pandas as pd
import numpy as np
import cv2
import collections
import time
from torchmetrics import Recall
from torchmetrics import Accuracy
from preprocess import *
import torchvision

In [10]:
def shuffle(*arrays, **kwargs):
    """This is not an inplace operation. Therefore, you can shuffle without worrying changing data."""
    if len(set(len(x) for x in arrays)) != 1:
        raise ValueError('All inputs to shuffle must have '
                         'the same length.')

    shuffle_indices = np.arange(len(arrays[0]))
    np.random.shuffle(shuffle_indices) # fix this for reproducible

    if len(arrays) == 1:
        return arrays[0][shuffle_indices]
    else:
        return tuple(x[shuffle_indices] for x in arrays)
def minibatch(*tensors, **kwargs):

    batch_size = kwargs['batch_size']

    if len(tensors) == 1:
        tensor = tensors[0]
        for i in range(0, len(tensor), batch_size):
            yield tensor[i:i + batch_size]
    else:
        for i in range(0, len(tensors[0]), batch_size):
            yield tuple(x[i:i + batch_size] for x in tensors)

In [2]:
new_label={
    0:0,
    1:1,
    2:1,
    3:2,
    4:2
        }

In [3]:
PATH = "data/train/"
xdata = collections.defaultdict(list)
ytrain = []
for classes in [0,1,2,3,4]:
    ls =  os.listdir(PATH+str(classes))
    print(f"Processing images class: {classes}")
    for samples in ls:
        img = cv2.resize(cv2.imread(PATH+str(classes)+'/'+samples),(224,224))
        img = img.transpose((2, 0, 1))
        xdata[new_label[classes]].append(img)
        ytrain.append(new_label[classes])

Processing images class: 0
Processing images class: 1
Processing images class: 2
Processing images class: 3
Processing images class: 4


In [4]:
PATH = "data/test/"
tdata = collections.defaultdict(list)
ytest = []
for classes in [0,1,2,3,4]:
    ls =  os.listdir(PATH+str(classes))
    print(f"Processing images class: {classes}")
    for samples in ls:
        img = cv2.resize(cv2.imread(PATH+str(classes)+'/'+samples),(224,224))
        img = img.transpose((2, 0, 1))
        tdata[new_label[classes]].append(img)
        ytrain.append(new_label[classes])

Processing images class: 0
Processing images class: 1
Processing images class: 2
Processing images class: 3
Processing images class: 4


In [5]:
PATH = "data/val/"
vdata = collections.defaultdict(list)
yval = []
for classes in [0,1,2,3,4]:
    ls =  os.listdir(PATH+str(classes))
    print(f"Processing images class: {classes}")
    for samples in ls:
        img = cv2.resize(cv2.imread(PATH+str(classes)+'/'+samples),(224,224))
        img = img.transpose((2, 0, 1))
        vdata[new_label[classes]].append(img)
        yval.append(new_label[classes])

Processing images class: 0
Processing images class: 1
Processing images class: 2
Processing images class: 3
Processing images class: 4


In [6]:
Xtrain = xdata[0] + xdata[1] + xdata[2]
Ytrain = [0 for i in range(len(xdata[0]))] + [1 for i in range(len(xdata[1]))] + [2 for i in range(len(xdata[2]))]
Xtest = tdata[0] + tdata[1] + tdata[2]
Ytest = [0 for i in range(len(tdata[0]))] + [1 for i in range(len(tdata[1]))] + [2 for i in range(len(tdata[2]))]
Xval = vdata[0] + vdata[1] + vdata[2]
Yval = [0 for i in range(len(vdata[0]))] + [1 for i in range(len(vdata[1]))] + [2 for i in range(len(vdata[2]))]

In [7]:
Xtrain =torch.tensor(np.array(Xtrain),dtype=torch.float32)
Ytrain =torch.tensor(np.array(Ytrain),dtype=torch.long)
Xtest = torch.tensor(np.array(Xtest),dtype=torch.float32)
Ytest = torch.tensor(np.array(Ytest),dtype=torch.long)
Xval = torch.tensor(np.array(Xval),dtype=torch.float32)
Yval = torch.tensor(np.array(Yval),dtype=torch.long)
                    

In [94]:
Xtrain,Ytrain,Xtest,Ytest,Xval,Yval  = Preprocess_GD().pipeline()

Processing images class: 0
Processing images class: 1
Processing images class: 2
Processing images class: 3
Processing images class: 4
Processing images class: 0
Processing images class: 1
Processing images class: 2
Processing images class: 3
Processing images class: 4
Processing images class: 0
Processing images class: 1
Processing images class: 2
Processing images class: 3
Processing images class: 4


In [3]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import models
best_model_params_path = "best_model.pth"

class GDModel(nn.Module):

    def __init__(self, pretrained_model_name="resnet18", num_classes=3, freeze=True):
        super(GDModel, self).__init__()
        self.base_model = models.resnet18(pretrained=True)
        self.loss_func = nn.CrossEntropyLoss()
        self.optimizer = optim.Adam(self.parameters(), lr=0.001)
        self.accuracy = Accuracy(task="multiclass", num_classes=3)
        if freeze:
            for name,param in self.base_model.named_parameters():
                if 'fc' in name:
                    param.requires_grad = True
                else:
                    param.requires_grad = False

            num_features_in = self.base_model.fc.in_features
            self.base_model.fc = nn.Sequential(nn.Linear(num_features_in, num_classes),nn.Softmax(dim=1))

    def forward(self, x):
        x = self.base_model(x)
        return x
    def predict(self,x):
        self.train(False)
        outputs = self(x)
        y_preds = torch.argmax(outputs,dim=1)
        return y_preds
    def fitting(self,Xtrain,Ytrain, Xval,Yval ,num_epochs = 20):
        best_acc = 0.0
        t1 = time.time()
        Xtrain, Ytrain = shuffle(Xtrain,Ytrain)
        Xval, Yval = shuffle(Xval,Yval)
        
        for epoch in range(num_epochs):
            print(f'Epoch {epoch}/{num_epochs - 1}')
            print('-' * 10)
            self.train(True)
            epoch_loss = 0.0
            preds_epoch = torch.tensor([])
            for num,(batchX,batchY) in enumerate(minibatch(Xtrain,Ytrain,batch_size=32)):
                self.optimizer.zero_grad()
                outputs = self(batchX)
                loss = self.loss_func(outputs, batchY)
                loss.backward()
                self.optimizer.step()
                epoch_loss += loss.item()
                y_preds = torch.argmax(outputs,dim=1)
                preds_epoch = torch.cat([preds_epoch,y_preds], dim =0)
            epoch_acc = self.accuracy(preds_epoch, Ytrain)
            t2 = time.time()
            epoch_train_time = t2 - t1
            # valid
            acc = self.evaluate(Xval,Yval)
            if  epoch_acc > best_acc:
                best_acc = epoch_acc
                torch.save(self.state_dict(), best_model_params_path)
            print(f'Epoch{epoch} Training time:{epoch_train_time} Epoch_loss: {epoch_loss:.4f} Acc_train: {epoch_acc:.4f} Acc_valid: {acc:.4f}')
        acc_test = self.evaluate_test(Xtest,Ytest)
        print(f'Test Accuracy: {acc_test:.4f}')

    def evaluate(self,Xval,Yval):
        y_preds = self.predict(Xval)
        acc = self.accuracy(y_preds,Yval)
        return acc
    def evaluate_test(self,Xtest,Ytest):
        self.load_state_dict(torch.load(best_model_params_path))
        y_preds = self.predict(Xtest)
        acc = self.accuracy(y_preds,Ytest)
        return acc

        



In [1]:
from gd import *

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = GDModel()

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
preds = torch.tensor([])

In [19]:
outputs = model(Xtrain[:10])
output2 = model(Xtrain[10:20])

In [24]:
y_preds = torch.argmax(outputs,dim=1)

In [25]:
y_preds

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [22]:
outputs=torch.cat([outputs,output2],dim=0)

In [23]:
outputs.size()

torch.Size([20, 3])

In [111]:
y_preds = torch.argmax(outputs,dim=1)
y_preds

tensor([0, 0, 0, 1, 0, 0, 0, 0, 0, 0])

In [112]:
y_preds = model.predict(Xtrain[:10])
y_preds

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [26]:
predict = model(Xtrain[:3])
y_preds = torch.argmax(predict,dim=0)

In [32]:
predict

tensor([[0.4725, 0.3138, 0.2137],
        [0.3875, 0.4067, 0.2058],
        [0.5062, 0.3061, 0.1876]], grad_fn=<SoftmaxBackward0>)

In [31]:
y_pred

tensor([2, 1, 0])

In [36]:
accuracy = Accuracy(task="multiclass", num_classes=3)
epoch_acc = accuracy(y_preds, torch.tensor([2,1,0]))

In [37]:
epoch_acc

tensor(1.)

In [23]:
predict

tensor([[0.4725, 0.3138, 0.2137],
        [0.3875, 0.4067, 0.2058],
        [0.5062, 0.3061, 0.1876]], grad_fn=<SoftmaxBackward0>)

In [19]:
predict.size()

torch.Size([3, 3])

In [17]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

AttributeError: 'GDModel' object has no attribute 'fc'

In [1]:
from gd import *
gd_model = GDModel()

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [2]:
from preprocess import Preprocess_GD
process_gd = Preprocess_GD()
Xtrain,Ytrain,Xtest,Ytest,Xval,Yval  = process_gd.pipeline()

Ultralytics YOLOv8.2.2 🚀 Python-3.8.0 torch-2.2.1 CPU (Apple M1)
Setup complete ✅ (8 CPUs, 16.0 GB RAM, 274.6/460.4 GB disk)
Processing images class: 0
Processing images class: 1
Processing images class: 2
Processing images class: 3
Processing images class: 4
Processing images class: 0
Processing images class: 1
Processing images class: 2
Processing images class: 3
Processing images class: 4
Processing images class: 0
Processing images class: 1
Processing images class: 2
Processing images class: 3
Processing images class: 4


In [3]:
gd_model.fitting(Xtrain,Ytrain,Xval,Yval,Xtest,Ytest)

Epoch 0/19
----------


KeyboardInterrupt: 

In [2]:
# Configuration settings
class Config:
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    num_epochs = 50
    num_classes = 3
    learning_rate = 1e-4
    model = torchvision.models.resnet18(weights='IMAGENET1K_V1')
    lr_decay = 0.1
    patience = 5
    batch_size = 32
    criterion = nn.CrossEntropyLoss()

config = Config()

NameError: name 'torch' is not defined

In [3]:
model = config.model
model.fc = nn.Linear(model.fc.in_features, 3)

NameError: name 'config' is not defined

In [4]:
model.load_state_dict(torch.load("/Users/namle/Desktop/SegJSW/bestresnet.pt",map_location=torch.device('cpu')))

NameError: name 'model' is not defined

In [1]:
preds = model(Xtrain)

NameError: name 'model' is not defined

In [19]:
preds.size()

tensor([[ 0.6332,  0.3212, -1.8132]], grad_fn=<AddmmBackward0>)